# Actividad Práctica 2 <a class="tocSkip">
# Técnicas de aprendizaje profundo <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Redes-neuronales-convolucionales:-AlexNet" data-toc-modified-id="Redes-neuronales-convolucionales:-AlexNet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Redes neuronales convolucionales: AlexNet</a></span><ul class="toc-item"><li><span><a href="#Primera-capa" data-toc-modified-id="Primera-capa-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Primera capa</a></span></li><li><span><a href="#Actividad-1" data-toc-modified-id="Actividad-1-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Actividad 1</a></span></li><li><span><a href="#Actividad-2" data-toc-modified-id="Actividad-2-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Actividad 2</a></span></li><li><span><a href="#Actividad-3" data-toc-modified-id="Actividad-3-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Actividad 3</a></span></li><li><span><a href="#Actividad-4" data-toc-modified-id="Actividad-4-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Actividad 4</a></span></li><li><span><a href="#Actividad-5" data-toc-modified-id="Actividad-5-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Actividad 5</a></span></li><li><span><a href="#Actividad-6" data-toc-modified-id="Actividad-6-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Actividad 6</a></span></li><li><span><a href="#Actividad-7" data-toc-modified-id="Actividad-7-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Actividad 7</a></span></li></ul></li><li><span><a href="#Conclusiones" data-toc-modified-id="Conclusiones-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conclusiones</a></span></li></ul></div>

# Redes neuronales convolucionales: AlexNet

In [4]:
# Definicion de librerias con la funciones que seran utilizadas por Keras.
import keras
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D

## Primera capa

In [11]:
#Definicion de contenedor y primera capa de AlexNet.
modelAlexNet = Sequential()
modelAlexNet.add(ZeroPadding2D((2,2), input_shape=(224, 224, 3)))

# modelAlexNet.add(Convolution2D(96,11,11,subsam-ple=(4,4),border_mode='valid'))
# /home/diego/anaconda3/envs/tensor/lib/python3.6/site-packages/ipykernel/__main__.py:4: 
# UserWarning: Update your `Conv2D` call to the Keras 2 API: 
# `Conv2D(96, (11, 11), strides=(4, 4), padding="valid")`

modelAlexNet.add(Conv2D(96, (11,11), strides=(4,4), padding='valid'))
modelAlexNet.summary()
## aquí quedé


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 96)        34944     
Total params: 34,944
Trainable params: 34,944
Non-trainable params: 0
_________________________________________________________________


La primera capa tiene 34944 parámetros. Esto se debe a un mapeo de un subconjunto de puntos de tres dimensiones (RGB) en una matriz de  $\mathbb{R}^{11\times11}$, subconjunto de la matriz de tamaño $224\times224$

Esto significa que cada punto en la segunda capa es el resultado de la suma ponderada de $11\times11\times3$ pesos en la primera capa, haciendo un total de 363+1 puntos (incluyendo el bias). Considerando que son 96 filtros, se tiene, para la primera capa

$$ 364\times96 = 34944 \mbox{ parámetros}$$



In [12]:
# añadiendo capa de activación
modelAlexNet.add(Activation(activation='relu'))
modelAlexNet.add(BatchNormalization())
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 96)        384       
Total params: 35,328
Trainable params: 35,136
Non-trainable params: 192
_________________________________________________________________


La capa de normalización posee 384 parámetros. Keras implementa esta capa siguiendo [este paper](http://proceedings.mlr.press/v37/ioffe15.pdf) donde por cada una de las 96 capas, existen cuatro parámetros que deben ser calculados. Es así que la cantidad de parámetros de esta capa es $96\times4=384$

## Actividad 1

Considerando la dimensionalidad de la capa de entrada de $224\times224$,

*  Se hace un `ZeroPadding2D`, el que añade 2 filas a ambos costados, aumentando la dimensionalidad en 4 por eje. De aquí que la dimensión final sea de $228\times228$
*  Se hace un mapeo usando subconjuntos como filtros de tamaño $11\times11$, usando un *stride* de 4. De aquí que la dimensionalidad de la capa siguiente sea de

$$\frac{228}{4} = 57 $$

Dado que esta capa considera `padding = 'valid'`, se tiene que la última posición válida es múltiplo de 11, o dicho de otra forma $11\times5$, quedando las dos últimas filas/columnas fuera de la siguiente capa

In [13]:
print(modelAlexNet.output_shape)

(None, 55, 55, 96)


Considerando que `MaxPooling2D(3,3)` actúa para estos efectos como un mapeo de una matriz $M$ de $3\times3$ a un valor por cada dos columnas/filas de $M$, la dimensión resultante está dada por

$$\frac{55}{2} = 27.5 $$

Dado que 3 cabe 9 veces en 27, ésta es la dimensión resultante. Por lo tanto la estructura de la red *después* de `maxPooling2D`

In [16]:
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))
print(modelAlexNet.output_shape)

(None, 27, 27, 96)


## Actividad 2
La red hasta esta capa tiene la siguiente forma

In [17]:
print(modelAlexNet.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
Total params: 35,328
Trainable params: 35,136
Non-trainable params: 192
_________________________________________________________________
None


Si se agrega la segunda capa de `AlexNet`, la dimensionalidad de la red se modifica  de la siguiente manera:

In [26]:
# se agregan dos filas y columnas de ceros a cada lado de las capas
modelAlexNet.add(ZeroPadding2D((2,2)))

In [27]:
modelAlexNet.output_shape

(None, 31, 31, 96)

Se aplica convolución sobre las 96 capas anteriores, considerando un área de $5x5$ sobre cada una de las capas

In [33]:
# Se aplican 256 filtros sobre un área de 5x5
modelAlexNet.add(Conv2D(256, (5, 5), padding='valid'))

la dimensión resultante está dada por el `stride` por defecto de `(1,1)`, y por `valid` que no considera el `padding`, de tal manera que

In [37]:
modelAlexNet.output_shape

(None, 27, 27, 256)

In [38]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 27, 27, 256)       614656    
Total para

La cantidad de parámetros de esta capa se deben a

$$ (\mbox{ tamaño kernel }\times\mbox{ cantidad filtros capa anterior } +  \mbox{ bias })\times\mbox{ cantidad filtros capa actual }$$

$$ \left((5\times5)\times96 + 1\right)\times256 = 614656 $$


Se añade una capa de activación `relu`, que lleva a cero valores negativos y mantiene los positivos. En este caso la dimensionalidad se mantiene:

In [39]:
modelAlexNet.add(Activation(activation='relu'))

In [40]:
modelAlexNet.output_shape

(None, 27, 27, 256)

Se normalizan los datos, dejándolos con media 0 y desviación estándar 1. Las dimensiones deberían mantenerse:


In [41]:
# TODO: ver los valores y dimensiones en esta capa.
modelAlexNet.add(BatchNormalization())

In [42]:
modelAlexNet.output_shape

(None, 27, 27, 256)

In [43]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_4 (ZeroPaddin (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_2 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 27, 27, 256)       614656    
__________

Luego se añade una nueva capa de `MaxPooling` con un stride de 2. De igual manera, la dimensionalidad de cada capa se ve reducida de $27\times27$ a $27/2\approx13$

In [58]:
# stride de 2 reduce la dimensionalidad de 27/2 approx 13
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

In [59]:
modelAlexNet.output_shape

(None, 13, 13, 256)

la capa resultante es el resultado de todas estas acciones, las que se resumen de la siguiente manera

In [97]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

## Actividad 3
La tercera capa se construye bajo las siguientes consideraciones 

*  Se incluye un *padding* de 1 cero a cada lado del mapa de activaciones de entrada
*  No utiliza normalización *batch*
*  No se considera `MaxPooling`

Para la primera parte, el mapa de activaciones de entrada viene dado por el resultado del `MaxPooling` anterior

In [99]:
modelAlexNet.output_shape

(None, 13, 13, 256)

Donde el mapa es de dimensión $13\times13$. Se añade un padding de 1 cero en ambas direcciones 

In [100]:
modelAlexNet.add(ZeroPadding2D((1,1)))

In [101]:
modelAlexNet.output_shape

(None, 15, 15, 256)

Se agrega una capa de convolución de 384 filtros, con un tamaño de filtro de $3\times3$

In [102]:
modelAlexNet.add(Conv2D(384, (3, 3), padding='valid'))

In [103]:
modelAlexNet.output_shape

(None, 13, 13, 384)

La capa 3 finalmente queda definida por

In [107]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

## Actividad 4
A continuación se definen las capas 4 y 5 siguientes. Las consideraciones son las mismas que las para la capa 3, esto es

*  Para la capa 4
    *  Se incluye un *padding* de 1 cero a cada lado del mapa de activaciones de entrada
    *  No utiliza normalización *batch*
    *  No se considera `MaxPooling`
*  Para la capa 5
    *  Se incluye un *padding* de 1 cero a cada lado del mapa de activaciones de entrada
    *  No utiliza normalización *batch*
    *  Si considera `MaxPooling`, con una ventana de $3\times3$ y un stride de 2

In [108]:
# Capa 4
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Conv2D(384, (3, 3), padding='valid'))
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

In [109]:
# capa 5
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Conv2D(256, (3, 3), padding='valid'))
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

## Actividad 5
Esta actividad se refiere a las últimas capas de `AlexNet`, las cuales son densas. El primer paso es convertir la matriz de pesos de la capa 5 de $13\times13$ ($6\times6$ luego del `Maxpooling`) a una estructura plana unidimensional. 

In [117]:
modelAlexNet.add(Flatten())

In [118]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

## Actividad 6

La capa 6 se obtiene entonces
1.  Mapeando la versión *plana* de la capa 5 sobre una capa densa de 4096 unidades
1.  Añadiendo una capa de activación que use una función `ReLu`. Esto no cambia la dimensionalidad de la capa.
1.  Añadiendo un `Dropout` de 0.5, lo que tampoco cambia la dimensionalidad de la capa

In [123]:
modelAlexNet.add(Dense(input_dim=9216, units=4096))

In [125]:
modelAlexNet.output_shape

(None, 4096)

In [126]:
modelAlexNet.add(Activation(activation='relu'))

In [127]:
modelAlexNet.output_shape

(None, 4096)

In [128]:
modelAlexNet.add(Dropout(0.5))

In [129]:
modelAlexNet.output_shape

(None, 4096)

In [130]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

## Actividad 7
Las capas a continuación tienen las siguientes características

*  Capa 7
    *  Es una capa densa de 4096 unidades que acepta las entradas de la capa anterior
    *  Añade una capa de activación `ReLu` y otra de `Dropout` a la salida


In [131]:
modelAlexNet.add(Dense(input_dim=4096, units=4096))

In [132]:
modelAlexNet.add(Activation(activation='relu'))

In [133]:
modelAlexNet.add(Dropout(0.5))

*  Capa 8
    *  Capa densa de 1000 unidades que tiene una función activación `SoftMax`

In [134]:
modelAlexNet.add(Dense(1000))

In [135]:
modelAlexNet.add(Activation('softmax'))

In [136]:
modelAlexNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_10 (ZeroPaddi (None, 228, 228, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 256)       614656    
__________

# Conclusiones

En este problema se construyó la red `AlexNet` como se indica en la figura. La arquitectura requiere una serie de transformaciones sobre el espacio de datos de entrada del problema (imágenes en RGB) a un espacio más pequeño, aprendiendo más *matices* o características en el proceso. 

![alex](img/alex.png)

*  Las capas densas aumentan considerablemente la cantidad de parámetros, debido a que son *fully connected*. 
*  El *max pooling* permite reducir la dimensión del espacio de búsqueda de la imagen
*  La convolución permite añadir más pesos a una capa sin necesidad de transformar el espacio de búsqueda. Dicho de otras palabras, permite aprender una cantidad determinada de características sobre este espacio.


